In [226]:
import pymongo 
from datetime import datetime
from pymongo import MongoClient
from bson.objectid import ObjectId
from pprint import pprint as pp

In [227]:
# connect to database
connection = MongoClient('localhost', 27017)

db = connection.sample


In [228]:
# handle to names collection
towns = db.towns
towns.drop()

In [229]:
town = {
"name": "New York",
"population": 22200000,
"last_census": datetime(2007, 7, 31),
"famous_for": [ "statue of liberty", "food" ],
"mayor" : {
    "name" : "Michael Bloomberg",
    "party" : "I"
    }
}

In [230]:
towns.insert_one(town)

In [231]:
for town in towns.find():
    print town

{u'name': u'New York', u'mayor': {u'party': u'I', u'name': u'Michael Bloomberg'}, u'last_census': datetime.datetime(2007, 7, 31, 0, 0), u'_id': ObjectId('5620b0c9801d3209ce58f022'), u'famous_for': [u'statue of liberty', u'food'], u'population': 22200000}


In [232]:
def insertCity(name, population, last_census, famous_for, mayor_info):
    town = {
        "name": name,
        "population": population,
        "last_census": datetime.strptime(last_census, '%Y-%d-%m'),
        "famous_for": famous_for,
        "mayor" : mayor_info
        }
    return str(towns.insert_one(town).inserted_id)

In [233]:
insertCity("Punxsutawney", 6200, '2008-31-01',
            ["phil the groundhog"], { "name" : "Jim Wehrle" }
)

'5620b0cc801d3209ce58f023'

In [234]:
town_id = insertCity("Portland", 582000, '2007-20-09',
["beer", "food"], { "name" : "Sam Adams", "party" : "D" }
)

In [235]:
for town in db.towns.find({ "_id" : ObjectId(town_id) }):
    pp(town)

{u'_id': ObjectId('5620b0cd801d3209ce58f024'),
 u'famous_for': [u'beer', u'food'],
 u'last_census': datetime.datetime(2007, 9, 20, 0, 0),
 u'mayor': {u'name': u'Sam Adams', u'party': u'D'},
 u'name': u'Portland',
 u'population': 582000}


In [238]:
for town in db.towns.find( {"_id" : ObjectId(town_id) }, { "name" : 1, "_id" : 0 }):
    pp(town)

{u'name': u'Portland'}


In [239]:
for town in db.towns.find( {"_id" : ObjectId(town_id) }, { "name" : 0 }):
    pp(town)

{u'_id': ObjectId('5620b0cd801d3209ce58f024'),
 u'famous_for': [u'beer', u'food'],
 u'last_census': datetime.datetime(2007, 9, 20, 0, 0),
 u'mayor': {u'name': u'Sam Adams', u'party': u'D'},
 u'population': 582000}
